In [ ]:
import numpy as np
import pandas as pd
import random
import csv
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.path.dirname('../NEUROEVOLUTION'), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    
from neat_utils import neat_cross_validation
from NEUROEVOLUTION.utils import drop_features, kfold, save_best_params, seed

target = 'fat'


In [ ]:
X = pd.read_csv('../datamart/data_project_nel.csv')
X = drop_features(X)
y = pd.read_csv('../datamart/y_' + f'{target}'+ '.csv')[f'{target}'+'_percent']

In [ ]:
n_iters = [25, 50, 75, 100]
best_score = np.inf
name = 'logs/' +f'{target}' + '/random_search_total' + '.csv'

with open(name, 'a', newline='\n') as csvfile:
        w = csv.writer(csvfile, delimiter=';')
        w.writerow(['i'] + ['n_iter'] + ['config_file']+ ['score'])


for i in range(1,50):
    config_n = random.choice([1,2,3,4,5,6])
    config_file = f'configs/config_{config_n}'
    
    n_iter = random.choice(n_iters)
    
    log_path_cv = 'logs/' + f'{target}'+'/' + f'{i}' + '_cv_results.csv'
    log_path_train = 'logs/'+ f'{target}'+ '/train_logs.csv'
    
    
    score = neat_cross_validation(X, y, config_file, kfold, n_iter, train_log_path = log_path_train, cv_log_path = log_path_cv, id=i)
    
    with open(name, 'a', newline='\n') as csvfile:
        w = csv.writer(csvfile, delimiter=';')
        w.writerow([i] + [n_iter] + [config_file]+ [score])
    
    if score < best_score:
        best_score = score
        print(f'New best cv_score: {best_score} at search {i}')
        best = score
        best_config = config_file
        best_iter = n_iter
        best_params = {
            'X': X,
            'y': y,
            'n_iter': n_iter,
            'config_file': config_file,
        }
        save_best_params('best_params/'+ f'{target}' + '-best_params-' + f'{i}' +'.pkl', best_params)

save_best_params('best_params/'+ f'{target}' + '-best_params_final.pkl', best_params)